In [1]:
import torch

import flash
from flash.audio import AudioClassificationData
from flash.image import ImageClassifier

/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


In [4]:
from flash.core.data.utils import download_data

download_data("https://pl-flash-data.s3.amazonaws.com/urban8k_images.zip", "/workspaces/audio-ml/data/i")


/workspaces/audio-ml/data/i/urban8k_images.zip:   0%|          | 0/225669 [00:00<?, ?KB/s]

In [5]:
datamodule = AudioClassificationData.from_folders(
    train_folder="/workspaces/audio-ml/data/i/urban8k_images/train",
    val_folder="/workspaces/audio-ml/data/i/urban8k_images/val",
    transform_kwargs=dict(spectrogram_size=(64, 64)),
    batch_size=4,
)

In [6]:
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [7]:
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy=("freeze_unfreeze", 1))

/usr/local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /workspaces/audio-ml/src/lightning_logs

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 11.2 M
-------------------------------------------------
14.7 K    Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/site-packages/pytorch_lightning/callbacks/finetuning.py:211: UserWarning: The provided params to be frozen already exist within another group of this optimizer. Those parameters will be skipped.
HINT: Did you init your optimizer in `configure_optimizer` as such:
 <class 'torch.optim.adam.Adam'>(filter(lambda p: p.requires_grad, self.parameters()), ...) 
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [11]:
datamodule = AudioClassificationData.from_files(
    predict_files=[
        "/workspaces/audio-ml/data/i/urban8k_images/test/air_conditioner/13230-0-0-5.wav.jpg",
        "/workspaces/audio-ml/data/i/urban8k_images/test/children_playing/9223-2-0-15.wav.jpg",
        "/workspaces/audio-ml/data/i/urban8k_images/test/jackhammer/22883-7-10-0.wav.jpg",
    ],
    transform_kwargs=dict(spectrogram_size=(64, 64)),
    batch_size=4,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)


Predicting: 1575it [00:00, ?it/s]

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[3, 217, 223, 3] to have 3 channels, but got 217 channels instead

In [8]:
trainer.save_checkpoint("audio_classification_model.pt")


In [10]:
datamodule